In [ ]:
from tensorflow.keras.models import Sequential
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import RandomFlip, RandomRotation, RandomZoom, RandomContrast
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.applications import ResNet50V2

In [2]:
data_dir = os.path.join('/kaggle/input/rice-pistachio-and-grapevine-leaf-classification/train', 'train')
df = pd.read_csv("/kaggle/input/rice-pistachio-and-grapevine-leaf-classification/train.csv")

In [3]:
def load_image(image_path, label):
    img = tf.io.read_file(image_path)
    img = tf.io.decode_image(img, channels=3)
    img.set_shape([None, None, 3])
    img = tf.image.resize(img, (256, 256))
    img = img/255.0

    return img, label

In [4]:
class_names = df['TARGET'].unique()
class_to_int = {name: i for i, name in enumerate(class_names)}
int_to_class = {i: name for i, name in enumerate(class_names)}

In [ ]:
full_paths = [os.path.join(data_dir, fname) for fname in df['ID'].values]
label = df['TARGET'].map(class_to_int).values
dataset = tf.data.Dataset.from_tensor_slices((full_paths, label))
dataset = dataset.map(load_image)
dataset = dataset.shuffle(buffer_size=len(full_paths))

In [6]:
BATCH_SIZE = 32
num_images = 6400
train_size = int(num_images * 0.8)
val_size = int(num_images * 0.2)
train = dataset.take(train_size)
val = dataset.skip(train_size).take(val_size)

train = train.batch(BATCH_SIZE)
val = val.batch(BATCH_SIZE)

In [7]:
data_augmentation = Sequential([
    RandomFlip("horizontal"),
    RandomRotation(0.2),
    RandomZoom(0.2),
    RandomContrast(0.2)
], name="data_augmentation")

In [8]:
pre_trained_model = ResNet50V2(
    include_top=False,
    weights="imagenet",
    input_shape=(256,256,3),
    pooling='avg'
)

pre_trained_model.trainable = True
for layer in pre_trained_model.layers[:-20]:
    layer.trainable = False

94668760/94668760 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [9]:
model = Sequential([
    data_augmentation,
    pre_trained_model,
    Dense(512, activation='relu'),
    Dropout(0.4),
    Dense(20, activation='softmax')
])

In [10]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ data_augmentation (Sequential)  │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ resnet50v2 (Functional)         │ (None, 2048)           │    23,564,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,564,800 (89.89 MB)

 Trainable params: 12,084,224 (46.10 MB)

 Non-trainable params: 11,480,576 (43.79 MB)

In [11]:
model.compile(
    optimizer=tf.keras.optimizers.AdamW(learning_rate=1e-4),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [12]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=2
)

In [13]:
resnet = model.fit(
    train,
    epochs=50,
    validation_data=val,
    callbacks=[early_stopping, reduce_lr]
)

Epoch 1/50


I0000 00:00:1759493974.162583      58 cuda_dnn.cc:529] Loaded cuDNN version 90300


160/160 ━━━━━━━━━━━━━━━━━━━━ 67s 210ms/step - accuracy: 0.5511 - loss: 1.4154 - val_accuracy: 0.8844 - val_loss: 0.3261 - learning_rate: 1.0000e-04
Epoch 2/50
160/160 ━━━━━━━━━━━━━━━━━━━━ 38s 190ms/step - accuracy: 0.8430 - loss: 0.4504 - val_accuracy: 0.9172 - val_loss: 0.2459 - learning_rate: 1.0000e-04
Epoch 3/50
160/160 ━━━━━━━━━━━━━━━━━━━━ 39s 191ms/step - accuracy: 0.8905 - loss: 0.3104 - val_accuracy: 0.9414 - val_loss: 0.1708 - learning_rate: 1.0000e-04
Epoch 4/50
160/160 ━━━━━━━━━━━━━━━━━━━━ 38s 191ms/step - accuracy: 0.9147 - loss: 0.2464 - val_accuracy: 0.9328 - val_loss: 0.1864 - learning_rate: 1.0000e-04
Epoch 5/50
160/160 ━━━━━━━━━━━━━━━━━━━━ 38s 191ms/step - accuracy: 0.9251 - loss: 0.2180 - val_accuracy: 0.9609 - val_loss: 0.1319 - learning_rate: 1.0000e-04
Epoch 6/50
160/160 ━━━━━━━━━━━━━━━━━━━━ 38s 188ms/step - accuracy: 0.9353 - loss: 0.1887 - val_accuracy: 0.9406 - val_loss: 0.1650 - learning_rate: 1.0000e-04
Epoch 7/50
160/160 ━━━━━━━━━━━━━━━━━━━━ 38s 189ms/step - 

In [14]:
test_dir = '/kaggle/input/rice-pistachio-and-grapevine-leaf-classification/test/test'
test_filepaths = sorted([os.path.join(test_dir, fname) for fname in os.listdir(test_dir)])

def load_test(filepath):
    img = tf.io.read_file(filepath)
    img = tf.io.decode_image(img, channels=3)
    img.set_shape([None, None, 3])
    img = tf.image.resize(img, (256, 256))
    img = img / 255.0
    return img

test_ds = tf.data.Dataset.from_tensor_slices(test_filepaths)
test_ds = test_ds.map(load_test).batch(32)

predictions = model.predict(test_ds, verbose=1)
predicted_class_indices = np.argmax(predictions, axis=1)
predicted_labels = [int_to_class[i] for i in predicted_class_indices]
file_names = [os.path.basename(p) for p in test_filepaths]
submission_df = pd.DataFrame({
    'ID': file_names,
    'TARGET': predicted_labels
})

submission_df.to_csv('submission.csv', index=False)
submission_df.head()

50/50 ━━━━━━━━━━━━━━━━━━━━ 6s 76ms/step


,ID,TARGET
0,0000.jpg,KARACADAG
1,0001.jpg,BRRI67
2,0002.jpg,BINADHAN16
3,0003.jpg,BINADHAN16
4,0004.jpg,KARACADAG
